## Data

This dataset was originally posted on Kaggle. **The key task is to predict whether a product/part will go on backorder.**

Product backorder may be the result of strong sales performance (e.g. the product is in such a high demand that production cannot keep up with sales). However, backorders can upset consumers, lead to canceled orders and decreased customer loyalty. Companies want to avoid backorders, but also avoid overstocking every product (leading to higher inventory costs).

This dataset has ~1.9 million observations of products/parts in an 8 week period. The source of the data is unreferenced.

* __Outcome__: whether the product went on backorder
* __Predictors__: Current inventory, sales history, forecasted sales, recommended stocking amount, product risk flags etc. (22 predictors in total)

The features and the target variable of the dataset are as follows:

**Description**
~~~
# Features: 
sku - Random ID for the product
national_inv - Current inventory level for the part
lead_time - Transit time for product (if available)
in_transit_qty - Amount of product in transit from source
forecast_3_month - Forecast sales for the next 3 months
forecast_6_month - Forecast sales for the next 6 months
forecast_9_month - Forecast sales for the next 9 months
sales_1_month - Sales quantity for the prior 1 month time period
sales_3_month - Sales quantity for the prior 3 month time period
sales_6_month - Sales quantity for the prior 6 month time period
sales_9_month - Sales quantity for the prior 9 month time period
min_bank - Minimum recommend amount to stock
potential_issue - Source issue for part identified
pieces_past_due - Parts overdue from source
perf_6_month_avg - Source performance for prior 6 month period
perf_12_month_avg - Source performance for prior 12 month period
local_bo_qty - Amount of stock orders overdue
deck_risk - Part risk flag
oe_constraint - Part risk flag
ppap_risk - Part risk flag
stop_auto_buy - Part risk flag
rev_stop - Part risk flag

# Target 
went_on_backorder - Product actually went on backorder
~~~

##  Overview / Roadmap

**General steps**:
* Part I: Preprocessing
  * Dataset carpentry & Exploratory Data Analysis
    * Development of functions to perform the necessary steps, data carpentry the Training and the Testing data.
  * Generated a **smart sample** of the the data
* Part II: Training and Validation
  * Created 3 alternative pipelines, each does:
      * Anomaly detection
      * Dimensionality reduction
      * Classification
* Part III: Testing
  * Train chosen model full training data
  * Evaluate model against testing
  * Write a summary of your processing and an analysis of the model performance

---

# Part I: Data Preprocessing

In this part, we preprocess the given training set. 


In [95]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd

from sklearn.utils import resample

## Load dataset

**Description**
~~~
sku - Random ID for the product
national_inv - Current inventory level for the part
lead_time - Transit time for product (if available)
in_transit_qty - Amount of product in transit from source
forecast_3_month - Forecast sales for the next 3 months
forecast_6_month - Forecast sales for the next 6 months
forecast_9_month - Forecast sales for the next 9 months
sales_1_month - Sales quantity for the prior 1 month time period
sales_3_month - Sales quantity for the prior 3 month time period
sales_6_month - Sales quantity for the prior 6 month time period
sales_9_month - Sales quantity for the prior 9 month time period
min_bank - Minimum recommend amount to stock
potential_issue - Source issue for part identified
pieces_past_due - Parts overdue from source
perf_6_month_avg - Source performance for prior 6 month period
perf_12_month_avg - Source performance for prior 12 month period
local_bo_qty - Amount of stock orders overdue
deck_risk - Part risk flag
oe_constraint - Part risk flag
ppap_risk - Part risk flag
stop_auto_buy - Part risk flag
rev_stop - Part risk flag
went_on_backorder - Product actually went on backorder. 
~~~

**Note**: This is a real-world dataset without any preprocessing.  
There will also be warnings due to fact that the 1st column is mixing integer and string values.  
**NOTE:** The last column, `went_on_backorder`, is what we are trying to predict.


In [96]:
# Dataset location
DATASET = 'Kaggle_Training_Dataset_v2.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)

dataset.head().transpose()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
sku,2889798,2922698,1215672,3101593,2896930
national_inv,0.0,886.0,355.0,3.0,10.0
lead_time,NaN,8.0,2.0,8.0,9.0
in_transit_qty,0.0,84.0,0.0,0.0,0.0
forecast_3_month,0.0,405.0,0.0,0.0,0.0
forecast_6_month,0.0,925.0,0.0,0.0,0.0
forecast_9_month,0.0,1341.0,120.0,0.0,0.0
sales_1_month,0.0,174.0,2.0,0.0,0.0
sales_3_month,0.0,532.0,17.0,0.0,1.0
sales_6_month,0.0,936.0,121.0,0.0,1.0


In [97]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
national_inv,1687860.0,496.111782,29615.233831,-27256.0,4.00,15.00,80.00,12334404.0
lead_time,1586967.0,7.872267,7.056024,0.0,4.00,8.00,9.00,52.0
in_transit_qty,1687860.0,44.052022,1342.741731,0.0,0.00,0.00,0.00,489408.0
forecast_3_month,1687860.0,178.119284,5026.553102,0.0,0.00,0.00,4.00,1427612.0
forecast_6_month,1687860.0,344.986664,9795.151861,0.0,0.00,0.00,12.00,2461360.0
forecast_9_month,1687860.0,506.364431,14378.923562,0.0,0.00,0.00,20.00,3777304.0
sales_1_month,1687860.0,55.926069,1928.195879,0.0,0.00,0.00,4.00,741774.0
sales_3_month,1687860.0,175.025930,5192.377625,0.0,0.00,1.00,15.00,1105478.0
sales_6_month,1687860.0,341.728839,9613.167104,0.0,0.00,2.00,31.00,2146625.0
sales_9_month,1687860.0,525.269701,14838.613523,0.0,0.00,4.00,47.00,3205172.0


## Processing

In this section, the goal is to figure out:

* which columns we can use directly,  
* which columns are usable after some processing,  
* and which columns are not processable or obviously irrelevant (like product id) that we will discard.

Then process and prepare this dataset for creating a predictive model.

In [98]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Data columns (total 23 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   sku                1687861 non-null  object 
 1   national_inv       1687860 non-null  float64
 2   lead_time          1586967 non-null  float64
 3   in_transit_qty     1687860 non-null  float64
 4   forecast_3_month   1687860 non-null  float64
 5   forecast_6_month   1687860 non-null  float64
 6   forecast_9_month   1687860 non-null  float64
 7   sales_1_month      1687860 non-null  float64
 8   sales_3_month      1687860 non-null  float64
 9   sales_6_month      1687860 non-null  float64
 10  sales_9_month      1687860 non-null  float64
 11  min_bank           1687860 non-null  float64
 12  potential_issue    1687860 non-null  object 
 13  pieces_past_due    1687860 non-null  float64
 14  perf_6_month_avg   1687860 non-null  float64
 15  perf_12_month_avg  1687860 non-n

### Take samples and examine the dataset

In [99]:
dataset.iloc[:3,:6]

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month
0,2889798,0.0,NaN,0.0,0.0,0.0
1,2922698,886.0,8.0,84.0,405.0,925.0
2,1215672,355.0,2.0,0.0,0.0,0.0


In [100]:
dataset.iloc[:3,6:12]

,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank
0,0.0,0.0,0.0,0.0,0.0,14.0
1,1341.0,174.0,532.0,936.0,1352.0,168.0
2,120.0,2.0,17.0,121.0,136.0,5.0


In [101]:
dataset.iloc[:3,12:18]

,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk
0,No,0.0,-99.00,-99.00,0.0,No
1,No,0.0,1.00,0.93,0.0,No
2,No,0.0,0.99,0.97,0.0,Yes


In [102]:
dataset.iloc[:3,18:24]

,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,No,No,Yes,No,No
1,No,No,Yes,No,No
2,No,No,Yes,No,No


### Drop columns that are obviously irrelevant or not processable

In [103]:
# Add code below this comment  (Question #E101)
# ----------------------------------
# Dropped column: sku
dataset= dataset.iloc[:,1:23]

### Find unique values of string columns

Now try to make sure that these Yes/No columns really only contains Yes or No.  
If that's true, proceed to convert them into binaries (0s and 1s).

**Tip**: use [unique()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) function of pandas Series.

Example

~~~python
print('went_on_backorder', dataset['went_on_backorder'].unique())
~~~

In [104]:
# All the column names of these yes/no columns
yes_no_columns = list(filter(lambda i: dataset[i].dtype!=np.float64, dataset.columns))
print(yes_no_columns)

# Add code below this comment  (Question #E102)
# ----------------------------------
#finding wheather a column contain only yes/no

print("potential_issue:", dataset.potential_issue.unique())
print("deck_risk:", dataset.deck_risk.unique())
print("oe_constraint", dataset.oe_constraint.unique())
print("ppap_risk", dataset.ppap_risk.unique())
print("stop_auto_buy", dataset.stop_auto_buy.unique())
print("rev_stop", dataset.rev_stop.unique())
print("went_on_backorder:", dataset.went_on_backorder.unique())


['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']
potential_issue: ['No' 'Yes' nan]
deck_risk: ['No' 'Yes' nan]
oe_constraint ['No' 'Yes' nan]
ppap_risk ['No' 'Yes' nan]
stop_auto_buy ['Yes' 'No' nan]
rev_stop ['No' 'Yes' nan]
went_on_backorder: ['No' 'Yes' nan]


You may see **nan** also as possible values representing missing values in the dataset.

We fill them using most popular values, the [Mode](https://en.wikipedia.org/wiki/Mode_%28statistics%29) in Stats.

In [106]:
for column_name in yes_no_columns:
    mode = dataset[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    dataset[column_name].fillna(mode, inplace=True)

Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No


### Convert yes/no columns into binary (0s and 1s)

In [107]:
# Add code below this comment  (Question #E103)
# ----------------------------------
cat = dataset.select_dtypes(include = ['object']).columns
for col in cat:
    dataset[col].replace({'No': 0, 'Yes': 1}, inplace=True)
    dataset[col] = dataset[col].astype(int)

Now all columns should be either int64 or float64.

In [108]:
#checking if dataset still contains any null value 
dataset.isnull().sum()


national_inv              1
lead_time            100894
in_transit_qty            1
forecast_3_month          1
forecast_6_month          1
forecast_9_month          1
sales_1_month             1
sales_3_month             1
sales_6_month             1
sales_9_month             1
min_bank                  1
potential_issue           0
pieces_past_due           1
perf_6_month_avg          1
perf_12_month_avg         1
local_bo_qty              1
deck_risk                 0
oe_constraint             0
ppap_risk                 0
stop_auto_buy             0
rev_stop                  0
went_on_backorder         0
dtype: int64

In [111]:
# the lead time column contains 100894 null values
#replacing lead_time null values with mode
dataset['lead_time'].fillna(dataset['lead_time'].mode()[0], inplace=True)
dataset.isnull().sum()
#still we  have a few columns with null value

In [113]:
#removing those rows with null values
dataset=dataset.dropna()
dataset.info()
#now, the dataset is cleaned

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1687860 entries, 0 to 1687860
Data columns (total 22 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   national_inv       1687860 non-null  float64
 1   lead_time          1687860 non-null  float64
 2   in_transit_qty     1687860 non-null  float64
 3   forecast_3_month   1687860 non-null  float64
 4   forecast_6_month   1687860 non-null  float64
 5   forecast_9_month   1687860 non-null  float64
 6   sales_1_month      1687860 non-null  float64
 7   sales_3_month      1687860 non-null  float64
 8   sales_6_month      1687860 non-null  float64
 9   sales_9_month      1687860 non-null  float64
 10  min_bank           1687860 non-null  float64
 11  potential_issue    1687860 non-null  int64  
 12  pieces_past_due    1687860 non-null  float64
 13  perf_6_month_avg   1687860 non-null  float64
 14  perf_12_month_avg  1687860 non-null  float64
 15  local_bo_qty       1687860 non-n

### Smartly sample the data into a more manageable size for cross-fold validation in Grid Search

In [114]:
num_backorder = np.sum(dataset['went_on_backorder']==1)
print('backorder ratio:', num_backorder, '/', len(dataset), '=', num_backorder / len(dataset))

backorder ratio: 11293 / 1687860 = 0.006690720794378681


In [115]:
# Add code below this comment   (Question #E104) 
# ----------------------------------

#undersampling

backorder = len(dataset[dataset['went_on_backorder'] == 1]) #no of samples with a backorder
non_backorder_indices = dataset[dataset.went_on_backorder == 0].index  #finding indices of non backorder samples 
random_indices = np.random.choice(non_backorder_indices,backorder, replace=False) #random sampling of non backorder indices
backorder_indices = dataset[dataset.went_on_backorder == 1].index  #indices of backorder samples
under_sample_indices = np.concatenate([backorder_indices,random_indices])  # concatinating both samples
under_sample = dataset.loc[under_sample_indices]  #Balance Dataframe


print(np.sum(under_sample['went_on_backorder']==1))
print(np.sum(under_sample['went_on_backorder']==0))

11293
11293


In [118]:
under_sample.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
34,0.0,12.0,0.0,48.0,84.0,120.0,7.0,45.0,81.0,130.0,...,0.0,0.98,0.89,0.0,0,0,0,1,0,1
399,5.0,4.0,0.0,1.0,3.0,6.0,0.0,4.0,7.0,9.0,...,0.0,0.89,0.90,0.0,0,0,0,1,0,1
834,0.0,8.0,0.0,8.0,8.0,12.0,1.0,2.0,6.0,6.0,...,0.0,0.78,0.78,0.0,0,0,0,1,0,1
1099,0.0,8.0,2.0,12.0,17.0,27.0,8.0,17.0,28.0,38.0,...,0.0,0.42,0.36,0.0,0,0,0,1,0,1
1115,6.0,8.0,0.0,19.0,19.0,30.0,4.0,15.0,32.0,61.0,...,18.0,0.18,0.31,0.0,0,0,0,1,0,1


In [119]:
# Write your smart sampling to local file  
# ----------------------------------
#saving the dataset for further use
import joblib
joblib.dump(under_sample, 'datasetnew.pkl')

['datasetnew.pkl']